In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import os

In [3]:
pred_path = '/content/drive/MyDrive/ModeloCompleto_Monografia/Eval_CompleteModel/output.csv'
gt_path = '/content/drive/MyDrive/red_neuronalSSD/dataset_SDSS/csv/formatoDetector/test.csv'
output_dir = '/content/drive/MyDrive/ModeloCompleto_Monografia'
file_name_dir = '/content/drive/MyDrive/ModeloCompleto_Monografia/Input_Detection/'

In [4]:
def calculate_iou(box_pred, box_gt):
    # Obtención de las coordenadas de las cajas predichas y de referencia
    obj_name, ori_name, xmin_pred, ymin_pred, xmax_pred, ymax_pred, detection_score, class_pred = box_pred
    orin_name_gt, h, w, class_gt, xmin_gt, ymin_gt, xmax_gt, ymax_gt = box_gt

    # Cálculo del área de intersección
    xmin_inter = max(xmin_pred, xmin_gt)
    ymin_inter = max(ymin_pred, ymin_gt)
    xmax_inter = min(xmax_pred, xmax_gt)
    ymax_inter = min(ymax_pred, ymax_gt)

    w_inter = max(0, xmax_inter - xmin_inter)
    h_inter = max(0, ymax_inter - ymin_inter)

    area_inter = w_inter * h_inter

    # Cálculo del área de unión
    area_pred = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    area_gt = (xmax_gt - xmin_gt) * (ymax_gt - ymin_gt)

    area_union = area_pred + area_gt - area_inter

    # Cálculo del IoU
    iou = area_inter / area_union if area_union > 0 else 0

    return iou

def calculate_iou_for_all_boxes(boxes_pred, boxes_gt):
    iou_matrix = []

    for box_pred in boxes_pred:
        iou_row = []

        for box_gt in boxes_gt:
            iou = calculate_iou(box_pred, box_gt)
            iou_row.append(iou)

        iou_matrix.append(iou_row)

    return iou_matrix
def find_best_match(iou_matrix):
    best_matches = []
    best_iou_matrix = []
    for i, row in enumerate(iou_matrix):
        max_iou = max(row)

        if max_iou > 0:
            best_match_index = row.index(max_iou)
        else:
            best_match_index = "Missed"

        best_matches.append(best_match_index)
        best_iou_matrix.append(max_iou)
    return best_matches, best_iou_matrix

In [5]:
file_list = os.listdir(file_name_dir)
boxes_pred = pd.DataFrame(pd.read_csv(pred_path)).to_numpy().tolist()
boxes_gt = pd.DataFrame(pd.read_csv(gt_path))
boxes_gt = boxes_gt[boxes_gt['filename'].isin(file_list)].to_numpy().tolist()

In [8]:
iou_matrix = calculate_iou_for_all_boxes(boxes_pred, boxes_gt)
best_matches, best_iou_matrix = find_best_match(iou_matrix)

In [9]:
# Crear una nueva lista para almacenar las predicciones actualizadas
boxes_pred_updated = []

# Iterar sobre las predicciones y los mejores índices correspondientes
for i, best_match_index in enumerate(best_matches):
    # Obtener las coordenadas de la caja de referencia correspondiente
    if best_match_index != "Missed":
        xmin_gt, ymin_gt, xmax_gt, ymax_gt = boxes_gt[best_match_index][4:8]
        class_gt = boxes_gt[best_match_index][3]
    else:
        # Si no hay una mejor coincidencia, asignar coordenadas vacías
        xmin_gt, ymin_gt, xmax_gt, ymax_gt = "", "", "", ""
        class_gt = ""
    iou_temp = best_iou_matrix[i]
    # Agregar las coordenadas de referencia a la predicción
    box_pred = boxes_pred[i]
    box_pred.extend([xmin_gt, ymin_gt, xmax_gt, ymax_gt,iou_temp,class_gt])

    # Agregar la predicción actualizada a la lista de predicciones actualizadas
    boxes_pred_updated.append(box_pred)


In [10]:
for box_pred in boxes_pred_updated:
    iou = box_pred[12]  # Índice de la columna que contiene el valor de IoU
    if iou > 0.3:
        box_pred.append("Verdadero Positivo")
    else:
        box_pred.append("Falso Positivo")
    if box_pred[7] == box_pred[13]:
        box_pred.append('Verdadero Positivo')
    else:
        box_pred.append("Falso Positivo")
    if box_pred[14] == 'Verdadero Positivo' and box_pred[15] == 'Verdadero Positivo':
        box_pred.append("Verdadero Positivo")
    else:
        box_pred.append("Falso Positivo")

In [11]:
boxes_pred_updated_df = pd.DataFrame(boxes_pred_updated,columns=['object_name','original_filename','xmin_pred','ymin_pred','xmax_pred','ymax_pred','detect_score','class_pred','xmin_gt','ymin_gt','xmax_gt','ymax_gt','iou','class_gt','eval_detect','eval_classification','eval_model'])

In [13]:
boxes_pred_updated_df.to_csv('/content/drive/MyDrive/ModeloCompleto_Monografia/comparacion.csv',index=False)

In [ ]:
# Contar los verdaderos positivos y falsos positivos en eval_detect
eval_detect_counts = boxes_pred_updated_df['eval_detect'].value_counts()
verdaderos_positivos_detect = eval_detect_counts['Verdadero Positivo']
falsos_positivos_detect = eval_detect_counts['Falso Positivo']

# Contar los verdaderos positivos y falsos positivos en eval_classification
eval_classification_counts = boxes_pred_updated_df['eval_classification'].value_counts()
verdaderos_positivos_classification = eval_classification_counts['Verdadero Positivo']
falsos_positivos_classification = eval_classification_counts['Falso Positivo']

# Contar los verdaderos positivos y falsos positivos en eval_model
eval_model_counts = boxes_pred_updated_df['eval_model'].value_counts()
verdaderos_positivos_model = eval_model_counts['Verdadero Positivo']
falsos_positivos_model = eval_model_counts['Falso Positivo']


In [ ]:
boxes_gt_df = pd.DataFrame(boxes_gt,columns=['filename','width','height','class','xmin_gt','ymin_gt','xmax_gt','ymax_gt'])

In [ ]:
valores_unicos = set(best_matches)
valores_unicos_filtrados = [x for x in valores_unicos if x != 'Missed']
valores_unicos_ordenados = sorted(valores_unicos_filtrados, key=lambda x: float(x) if isinstance(x, str) and x != 'Missed' else float('inf'))
presicion_detect = verdaderos_positivos_detect / (verdaderos_positivos_detect+falsos_positivos_detect)
precision_class = verdaderos_positivos_classification / ((verdaderos_positivos_classification+falsos_positivos_classification)*presicion_detect)
precision_model = verdaderos_positivos_model / (verdaderos_positivos_model+falsos_positivos_model)
recuerdo = len(valores_unicos_ordenados) / len(boxes_gt_df)
iou_promedio = boxes_pred_updated_df['iou'].mean()
print(f'Resultados del modelo \n Modelo Detección: \n Precision: {presicion_detect} \n Recuerdo: {recuerdo} \n IoU Promedio: {iou_promedio} \n \n Modelo Clasificación: \n Precision: {precision_class} \n \n Ensamble Completo: \n Precision: {precision_model}')

Resultados del modelo 
 Modelo Detección: 
 Precision: 0.972904078549849 
 Recuerdo: 0.7036578900865877 
 IoU Promedio: 0.6031404783146831 
 
 Modelo Clasificación: 
 Precision: 0.5521591460456089 
 
 Ensamble Completo: 
 Precision: 0.5240747734138973
